# connectAppGen Web 

## 1. Application Settings

In [1]:
from IPython.display import display, HTML
from ipywidgets import widgets
from ui_element import connectParams

appName = widgets.Text(description="App Name:", value='', placeholder='such as m3sp')
appVersion = widgets.Text(description="Version:", value='1.0.0')
appAuthor = widgets.Text(description="Author:", value='', placeholder='Your Name')
appHeader = widgets.HBox([appName, appVersion, appAuthor])
display(appHeader)

## 2. Panel Parameters

In [2]:
panel = connectParams('Panel', paramType='panel', beginParam = 1, defaultParams=3)

## 3. Properties 

In [3]:
properties = connectParams('Properties', paramType='properties', beginParam=0,  defaultParams=3, maxParams = 50)

## 4. Actions
One Action is currently supported with the GUI Version(with Cancelation) 

In [4]:
actions = connectParams('Actions', paramType='actions', beginParam=0, defaultParams=1, maxParams = 2)

## 5. Scripts

In [5]:
testScriptOption = widgets.Checkbox(value=True, description='Test', disabled=True, indent=False)
resolveScriptOption = widgets.Checkbox(value=True, description='Resolve', disabled=False, indent=False)
pollScriptOption = widgets.Checkbox(value=False, description='Poll', disabled=False, indent=False)
templateOption = widgets.Dropdown(
                    options=[('Default', 'template'), ('Cylance', 'template3')],
                    value='template',
                    description='Base Code:',
                )
footer = widgets.HBox([testScriptOption, resolveScriptOption, pollScriptOption, templateOption])
display(footer)

## 5. Final Step - Generate Configuration

In [6]:
import yaml
from generator import generateApp 

global finalObj, firstTime
firstTime = True
finalObj = {}

btn_generate = widgets.Button(description="Generate App!")
out = widgets.HTML(
    value="",
    description='Results: ',
    layout={'border': '1px solid black'}
)

def on_gen_button_clicked(b):
    global finalObj, firstTime   
    out.value = ''
    result = ''
    if finalObj != {}:
        finalObj = {}
    
    if firstTime: 
        display(out)
        firstTime = False
    
    # Verifying App Settings
    if len(appName.value) != 0:
        finalObj['name'] = appName.value
    else: 
        out.value = "Error: You should fill the App Name field! Halting.."
        return 
    
    if len(appVersion.value) != 0:
        finalObj['version'] = appVersion.value
    else: 
        out.value = "Error: You should fill the App Version field! Halting.."
        return 
    
    if len(appAuthor.value) != 0:
        finalObj['author'] = appAuthor.value
        finalObj['testEnable'] = True
    else: 
        out.value = "Error: You should fill the Author field! Halting.."
        return
    
    # Verifying Panel Parameters
    finalObj['panels'] = {}
    finalObj['panels']['title'] = f"{finalObj['name'].upper()} Connection"
    finalObj['panels']['description'] = f"{finalObj['name'].upper()} Connection details"
    finalObj['panels']['fields'] = []
    
    if len(panel.panelParams) > 0:
        i=1
        for p in panel.panelParams:
            if p.children[0].value == '':
                out.value = f"Error: Panel Parameter {i} field Name needs to be filled! Halting.."
                return
            i+=1
            if p.children[1].value:
                finalObj['panels']['fields'].append({p.children[0].value: 'encrypted'})
            else: 
                finalObj['panels']['fields'].append(p.children[0].value)
                
    # Verifying Properties Parameters          
    finalObj['properties'] = []
    if len(properties.panelParams) >0:
        i=1
        for p in properties.panelParams: 
            if p.children[0].value == '':
                out.value = f"Error: Properties Parameter {i} field Name needs to be filled! Halting.."
                return
            i+=1
            op1 = p.children[1].value
            op2 = p.children[2].value
            op3 = p.children[3].value
            op4 = p.children[4].value
            if not(op1 or op2 or op3 or op4):
                finalObj['properties'].append(p.children[0].value)
            else: 
                if op3 and op4: 
                    out.value = f"Error: Cannot have List and type boolean fields enabled together! Halting.."
                    return
                    
                subObj = {}
                
                if op1: 
                    subObj['inventory'] = True
                    subObj['asset_portal'] = True
                
                if op2: 
                    subObj['track_change'] = True
                
                if op3: 
                    subObj['list'] = True
                
                if op4: 
                    subObj['type'] = 'boolean'
                    
                obj = {p.children[0].value: subObj}
                
                finalObj['properties'].append(obj)
                
    # Scripts Section
    finalObj['scripts'] = []
    
    if resolveScriptOption.value:
        finalObj['scripts'].append({'Resolve': 'properties'})
        
    if testScriptOption.value:
        finalObj['scripts'].append({'Test': {'test': True}})
        
    if pollScriptOption.value:
        finalObj['scripts'].append({'Poll': {'discovery': True}})
    
    finalObj['actions'] = []
            
    if len(actions.panelParams)==1:
        _action = actions.panelParams[0].children[0].children[0].value # main action name
        _action_internal= '_'.join(_action.lower().split())
        
        _undoAction = actions.panelParams[0].children[0].children[1].value
        
        if _action == '':
            out.value = f"Error: Cannot have Action without Action Name.. Halting.."
            return
            
        finalObj['scripts'].append({_action: {'action': _action_internal}})
        
        if _undoAction!= '':
            finalObj['scripts'].append({_undoAction: {'action': _action_internal, 'is_cancel': True}})
        
        _actionObj = {}
        
        _aoProps = {
            'description': f"{_action} Command",
            'ip_required': False,
            'threshhold_percentage': 1,
            'params': []
        }
        if actions.panelParams[0].children[1].children[0].children[0].value >0: 
            for i in range(actions.panelParams[0].children[1].children[0].children[0].value):
                if actions.actParams.panelParams[i].value == '':
                    out.value = f"Error: Cannot have Action Param without Name.. Halting.."
                    return
                _aoProps['params'].append(actions.actParams.panelParams[i].value)
        
        _actionObj[_action_internal] = _aoProps        
        finalObj['actions'].append(_actionObj)
            
        
    finalObj['logoFileName'] = 'logo.png' 
    finalObj['template'] = templateOption.value
    
    _appName_ = '_'.join(finalObj['name'].lower().split())
    _fileName = f"{_appName_}_{finalObj['version']}.yaml"
    out.value = f"Success.. Writing Yaml file to {_fileName} and working on Generating your App Skeleton."
    with open(_fileName, 'w') as file:
        documents = yaml.safe_dump(finalObj, file, explicit_start = True)
    generateApp(_fileName)
    
     

btn_generate.on_click(on_gen_button_clicked)
display(btn_generate)

Button(description='Generate App!', style=ButtonStyle())

In [7]:
HTML('''<script>
code_show=true;
function code_toggle() {if (code_show){ $('div.input').hide();} else { $('div.input').show();}
code_show = !code_show}
$( document ).ready(code_toggle);</script>
<form action="javascript:code_toggle()"><input type="submit" value="Toggle Raw Code - On/Off"></form>''')